In [266]:
import pandas as pd
import numpy as np
import json
import pandas as pd

# Set the max columns to max because the dataset has a lot of columns
pd.set_option('display.max_columns', None)

pd.set_option("display.max_rows", None)

In [267]:
def load_data(title):
  with open(title, encoding='utf-8') as f:
    return json.load(f)

In [268]:
pop_df = load_data('skyrim_population.json')

In [269]:
# Convert the data into dataframe and save it as csv
pop_df = pd.DataFrame(pop_df)
pop_df.to_csv('skyrim_population_raw.csv', index=False)

In [270]:
df = pd.read_csv('skyrim_population_raw.csv')

In [271]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1062 entries, 0 to 1061
Data columns (total 48 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Source             1062 non-null   object
 1   Name               1062 non-null   object
 2   Home City          544 non-null    object
 3   House              267 non-null    object
 4   Race               1049 non-null   object
 5   Gender             1049 non-null   object
 6   Level              1062 non-null   object
 7   Class              1049 non-null   object
 8   RefID              1028 non-null   object
 9   BaseID             1056 non-null   object
 10  Other Information  1062 non-null   bool  
 11  Health             1045 non-null   object
 12  Magicka            1044 non-null   object
 13  Stamina            1029 non-null   object
 14  Primary Skills     916 non-null    object
 15  Morality           1023 non-null   object
 16  Aggression         1023 non-null   object


In [272]:
pop_df.isnull().sum().sort_values(ascending=False)

Available            1060
Added by             1057
Horses               1056
Merchant §           1052
Merchant ‡           1051
Soul                 1049
Type                 1049
Species              1049
AdoptHF              1038
Stronghold           1026
StewardHF            1024
Respawns             1007
Training             1001
Race Details          997
Services              993
Follower              993
Marry                 990
Store                 946
Protected             941
Perks                 937
Home Town             937
Merchant              924
Buys                  916
Gold                  914
Sells                 903
Added by              876
House                 795
Essential             773
Class Details         665
Home City             518
Location              388
Primary Skills        146
Faction(s)             42
Voice Type             42
Morality               39
Aggression             39
Stamina                33
Health                 16
Magicka     

In [273]:
df = pop_df.copy()

In [274]:
# Delete Other Information Column
df.drop(columns=['Other Information'], inplace=True)
df.drop(columns=['Soul'], inplace=True)

In [275]:
len(df.loc[df.duplicated(subset=['Source'])])

45

In [276]:
df.drop_duplicates(keep='first', inplace=True)

In [277]:
for i, col in enumerate(df.columns):
    print(i, col)

0 Source
1 Name
2 Home City
3 House
4 Race
5 Gender
6 Level
7 Class
8 RefID
9 BaseID
10 Health
11 Magicka
12 Stamina
13 Primary Skills
14 Morality
15 Aggression
16 Voice Type
17 Faction(s)
18 Class Details
19 Essential
20 Location
21 Services
22 Training
23 Merchant
24 Gold
25 Sells
26 Buys
27 Race Details
28 Follower
29 Store
30 Added by
31 AdoptHF
32 Protected
33 Respawns
34 Perks
35 Marry
36 StewardHF
37 Merchant §
38 Merchant ‡
39 Available
40 Home Town
41 Species
42 Type
43 Horses
44 Added by
45 Stronghold


In [278]:
df.columns[30]


'Added\xa0by'

In [279]:
df.rename(columns={
    'Home\xa0City': 'Home City',
    'Added\xa0by': 'Added_by',
    'Home\xa0Town': 'Home Town'}, inplace=True)

In [280]:
# Llenamos valores nulos en la columna 'Merchant'
df['Merchant'] = df['Merchant'].fillna(df['Merchant §'])  # Rellenar con 'Merchant §'
df['Merchant'] = df['Merchant'].fillna(df['Merchant ‡'])  # Rellenar con 'Merchant ‡'

# Eliminamos las columnas redundantes ya que hemos combinado su información en 'Merchant'
df.drop(columns=['Merchant §', 'Merchant ‡'], inplace=True, errors='ignore')  # Ignorar si las columnas no existen

In [281]:
# Manejo de la columna 'Added by' de forma segura
if 'Added_by' in df.columns:
    # Rellenamos valores nulos en 'Added by' con los valores de 'Added_by'
    df['Added by'] = df['Added by'].fillna(df['Added_by'])  # Asignamos el resultado de fillna a 'Added by'
    # Eliminamos la columna 'Added_by' después de usarla
    df.drop(columns=['Added_by'], inplace=True, errors='ignore')  # Ignorar si la columna no![spinner](place)


In [282]:
# Llenamos valores nulos en la columna 'Home City'
if 'Home Town' in df.columns:
    df['Home City'] = df['Home City'].fillna(df['Home Town'])  # Rellenar con valores de 'Home Town'

if 'Stronghold' in df.columns:
    df['Home City'] = df['Home City'].fillna(df['Stronghold'])  # Rellenar con valores de 'Stronghold'

# Eliminamos las columnas 'Home Town' y 'Stronghold' si existen
df.drop(columns=['Home Town', 'Stronghold'], inplace=True, errors='ignore')  # Ignorar si las columnas no existen


In [283]:
# Llenamos valores nulos en la columna 'Race'
if 'Type' in df.columns:
    df['Race'] = df['Race'].fillna(df['Type'])  # Rellenar valores nulos en 'Race' con los valores de 'Type'

# Asignamos 'Undead' para valores en blanco en la columna 'Race'
df.loc[df['Race'] == '', 'Race'] = 'Undead'  

# Eliminamos columnas innecesarias, asegurando que existen antes de intentar eliminarlas
cols_to_drop = ['Type', 'Species']
df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True, errors='ignore')  # Ignora si las columnas no están presentes



In [284]:
df.reset_index(inplace=True, drop=True)

In [285]:
df.loc[df['Name'].str.contains('Ref')]  # Filtrar registros que contienen 'Ref' en la columna 'Name'

,Source,Name,Home City,House,Race,Gender,Level,Class,RefID,BaseID,Health,Magicka,Stamina,Primary Skills,Morality,Aggression,Voice Type,Faction(s),Class Details,Essential,Location,Services,Training,Merchant,Gold,Sells,Buys,Race Details,Follower,Store,AdoptHF,Protected,Respawns,Perks,Marry,StewardHF,Available,Horses,Added by
91,https://en.uesp.net/wiki/Skyrim:Torom,Torom (RefID: 000661A4),NaN,NaN,Animals,NaN,2,NaN,000661A4,0002F442,21,0,24,Sneak,NaN,NaN,NaN,LeftHandDaighresHouseFaction,NaN,NaN,Left Hand Mine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,https://en.uesp.net/wiki/Skyrim:Vigilance,Vigilance (RefID: 0009A7AB),Markarth,Markarth Stables,Animals,NaN,PC×1 (range=6-50),NaN,0009A7AB,0009A7AA,10+(PC-1)×10,0,20+(PC-1)×5,"Block, Sneak, Archery, One-handed",NaN,NaN,NaN,MarkarthStablesFaction; PlayerPotentialAnimalF...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,https://en.uesp.net/wiki/Skyrim:Gratian_Caerel...,Gratian Caerellius (RefID: xx020A2E),Raven Rock,NaN,Undead,NaN,1,NaN,xx020A2E,xx020A2D,0 (Dead),,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Raven Rock Mine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,https://en.uesp.net/wiki/Skyrim:Millius,Millius (RefID: xx020A48),Raven Rock,NaN,Undead,NaN,1,NaN,xx020A48,xx020A47,0 (Dead),,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Raven Rock Mine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274,https://en.uesp.net/wiki/Skyrim:Snippy,Snippy (RefID: xx033A4D),NaN,NaN,Animals,NaN,1,NaN,xx033A4D,xx033A47,5,4,25,NaN,NaN,NaN,NaN,Riften Fishery Faction; TownRiftenFaction,NaN,Yes,Riften Docks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fishing
508,https://en.uesp.net/wiki/Skyrim:Tiber,Tiber (RefID: 00023EF2),Windhelm,Hollyfrost Farm,Animals,NaN,2,NaN,00023EF2,00023EF1,21,0,24,Sneak,NaN,NaN,NaN,Creature Faction; DogFaction; Hollyfrost Farm ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
509,https://en.uesp.net/wiki/Skyrim:Ysgramor_(dog),Ysgramor (RefID: 00023EEE),Windhelm,Hollyfrost Farm,Animals,NaN,2,NaN,00023EEE,00023EED,21,0,24,Sneak,NaN,NaN,NaN,Creature Faction; DogFaction; Hollyfrost Farm ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
639,https://en.uesp.net/wiki/Skyrim:Stump,Stump (RefID: 0001E62B),NaN,NaN,Animals,NaN,2,NaN,0001E62B,0001E62A,21,0,24,Sneak,NaN,NaN,NaN,NaN,NaN,NaN,Riverwood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
723,https://en.uesp.net/wiki/Skyrim:Maximian_Axius,Maximian Axius (RefID: xx030C9F),NaN,NaN,Undead,NaN,1,NaN,xx030C9F,xx030C9E,0 (Dead),0,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fort Frostmoth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
743,https://en.uesp.net/wiki/Skyrim:Bran,Bran (RefID: xx01AA7B),NaN,NaN,Animals,NaN,PC×1 (range=10-25),NaN,xx01AA7B,xx01AA74,100+(PC-1)×10,0,100+(PC-1)×5,"Block, Sneak, Archery, One-handed",NaN,NaN,NaN,Creature Faction; CrimeFactionRift; DLC1Hunter...,NaN,NaN,Fort Dawnguard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Dawnguard


In [286]:
for i, row in enumerate(df['Name']):
    if row.find('Ref') != -1:  # Verifica si la cadena 'Ref' está presente en el nombre
        row = row.split(' (')[0]  # Separa la cadena en una tupla y toma la primera parte
        df.at[i, 'Name'] = row  # Asigna el valor limpio a la posición correspondiente del DataFrame

df.loc[df['Name'].str.contains('Ref')]  # Verificar si aún existen registros con 'Ref' en el nombre

,Source,Name,Home City,House,Race,Gender,Level,Class,RefID,BaseID,Health,Magicka,Stamina,Primary Skills,Morality,Aggression,Voice Type,Faction(s),Class Details,Essential,Location,Services,Training,Merchant,Gold,Sells,Buys,Race Details,Follower,Store,AdoptHF,Protected,Respawns,Perks,Marry,StewardHF,Available,Horses,Added by


In [287]:
df['Level (PC=10)'] = df['Level']

In [288]:
for i, row in enumerate(df['Name']):
    if row.find('Ref') != -1:  # Verifica si la cadena 'Ref' está presente en el nombre
        row = row.split(' (')[0]  # Separa la cadena en una tupla y toma la primera parte
        df.at[i, 'Name'] = row  # Asigna el valor limpio a la posición correspondiente del DataFrame

df.loc[df['Name'].str.contains('Ref')]  # Verificar si aún existen registros con 'Ref' en el nombre

,Source,Name,Home City,House,Race,Gender,Level,Class,RefID,BaseID,Health,Magicka,Stamina,Primary Skills,Morality,Aggression,Voice Type,Faction(s),Class Details,Essential,Location,Services,Training,Merchant,Gold,Sells,Buys,Race Details,Follower,Store,AdoptHF,Protected,Respawns,Perks,Marry,StewardHF,Available,Horses,Added by,Level (PC=10)


In [289]:
# Iteramos sobre cada fila del DataFrame en la columna 'Level (PC=10)'
for i, val in enumerate(df['Level (PC=10)']):
    # Verificamos si la cadena contiene la palabra 'PC'
    if val.find('PC') != -1:
        # Reemplazamos 'PC' con '10' y los símbolos '×' o 'x' con '*'
        val = val.replace('PC', '10').replace('×', '*').replace('x', '*')
        
        # Si la cadena también contiene 'range' o 'Range' (mayúsculas o minúsculas)
        if (val.find('range') != -1) or (val.find('Range') != -1): 
            # Dividimos la cadena en dos partes: el valor y el rango
            val, rang = val.split(' ')
            # Extraemos el rango (por ejemplo, "5-10") eliminando paréntesis
            rang = rang.split('=')[1].replace(')', '')
            # Convertimos el rango a números
            low, high = pd.to_numeric(rang.split('-'))
            
            # Evaluamos la expresión matemática (por ejemplo, "5+3")
            val = eval(val)
            
            # Si el valor evaluado es menor que el límite inferior, lo ajustamos al límite inferior
            if val <= low:
                val = low
            # Si el valor evaluado es mayor que el límite superior, lo ajustamos al límite superior
            elif val >= high:
                val = high
            
            # Guardamos el valor ajustado en el DataFrame
            df.at[i, 'Level (PC=10)'] = val
        
        # Si la cadena contiene 'ma*' (máximo)
        elif val.find('ma*') != -1:
            # Dividimos la cadena en el valor y el límite máximo
            val, high = val.split(' ')
            # Extraemos el límite máximo (por ejemplo, "ma*=10")
            high = int(high.split('=')[1].replace(')', ''))
            
            # Evaluamos la expresión matemática
            val = eval(val)
            
            # Si el valor evaluado es mayor que el límite máximo, lo ajustamos al límite máximo
            if val >= high:
                val = high
            
            # Guardamos el valor ajustado en el DataFrame
            df.at[i, 'Level (PC=10)'] = val
        
        # Si la cadena contiene 'min' (mínimo)
        elif val.find('min') != -1:
            # Dividimos la cadena en el valor y el límite mínimo
            val, low = val.split(' ')
            # Extraemos el límite mínimo (por ejemplo, "min=5")
            low = int(low.split('=')[1].replace(')', ''))
            
            # Evaluamos la expresión matemática
            val = eval(val)
            
            # Si el valor evaluado es menor que el límite mínimo, lo ajustamos al límite mínimo
            if val <= low:
                val = low
            
            # Guardamos el valor ajustado en el DataFrame
            df.at[i, 'Level (PC=10)'] = val
        
        # Si no hay condiciones especiales, simplemente evaluamos la expresión
        else:
            val = eval(val)
            df.at[i, 'Level (PC=10)'] = val
    
    # Si la cadena contiene 'Radiant'
    elif val.find('Radiant') != -1:
        # Extraemos el rango entre paréntesis (por ejemplo, "Radiant(5-10)")
        val = val.split(' ')[1].replace('(', '').replace(')', '')
        # Convertimos el rango a números
        low, high = pd.to_numeric(val.split('-'))
        # Calculamos el promedio del rango
        val = (high + low) / 2
        # Guardamos el promedio en el DataFrame
        df.at[i, 'Level (PC=10)'] = val
    
    # Si la cadena contiene 'Leveled'
    elif val.find('Leveled') != -1:
        # Extraemos el rango entre paréntesis (por ejemplo, "Leveled(5-10)")
        val = val.split(' ')[1].replace('(', '').replace(')', '')
        # Convertimos el rango a números
        low, high = pd.to_numeric(val.split('-'))
        # Calculamos el promedio del rango
        val = (high + low) / 2
        # Guardamos el promedio en el DataFrame
        df.at[i, 'Level (PC=10)'] = val
    
    # Para todos los demás casos
    else:
        try:
            # Intentamos convertir la cadena directamente a un número
            val = pd.to_numeric(val)
            df.at[i, 'Level (PC=10)'] = val
        except:
            # Si no se puede convertir, asumimos que es un rango (por ejemplo, "5-10")
            low, high = pd.to_numeric(val.split('-'))
            # Calculamos el promedio del rango
            val = (high + low) / 2
            # Guardamos el promedio en el DataFrame
            df.at[i, 'Level (PC=10)'] = val

In [290]:
import pandas as pd
import re

# Crear las nuevas columnas de salud, magia y resistencia
df['Health (PC=10)'] = df['Health']
df['Magicka (PC=10)'] = df['Magicka']
df['Stamina (PC=10)'] = df['Stamina']

def procesar_valor_celda(val, valor_nivel=None):
    """
    Procesa valores de celda con varios formatos y devuelve resultados numéricos.
    """
    if pd.isna(val):
        return None
        
    val = str(val).strip()
    
    if '(Dead)' in val:
        return 0
        
    if 'Radiant' in val:
        match = re.search(r'\((\d+)-(\d+)\)', val)
        if match:
            bajo, alto = map(float, match.groups())
            return (bajo + alto) / 2
            
    pc_minus_match = re.match(r'(\d+)-(\d+)\s*\((\d+)-0\.8\)(\d+(?:\.\d+)?)', val)
    if pc_minus_match:
        rango_bajo = float(pc_minus_match.group(1))
        rango_alto = float(pc_minus_match.group(2))
        valor_pc = float(pc_minus_match.group(3))
        multiplicador = float(pc_minus_match.group(4))
        valor_base = (rango_alto + rango_bajo) / 2
        valor_ajustado = (valor_pc - 0.8) * multiplicador
        return min(max(valor_base, valor_ajustado), rango_alto)
    
    range_match = re.match(r'(\d+)-(\d+)$', val)
    if range_match:
        bajo, alto = map(float, range_match.groups())
        return (bajo + alto) / 2
        
    nivel_fraccion_match = re.match(r'(\d+(?:\.\d+)?)\s*\((\d+)/(\d+)\)\s*\+\s*(\d+(?:\.\d+)?)', val)
    if nivel_fraccion_match and valor_nivel is not None:
        base = float(nivel_fraccion_match.group(1))
        num = int(nivel_fraccion_match.group(2))
        den = int(nivel_fraccion_match.group(3))
        suma = float(nivel_fraccion_match.group(4))
        valor_fraccion = (num / den)
        return base * valor_fraccion + suma
        
    pc_match = re.match(r'(\d+)-(\d+)\s*\(((?:N)?PC)\)(\d+(?:\.\d+)?)', val)
    if pc_match:
        rango_bajo = float(pc_match.group(1))
        rango_alto = float(pc_match.group(2))
        tipo_pc = pc_match.group(3)
        multiplicador = float(pc_match.group(4))
        valor_base = 10 if tipo_pc == 'PC' else valor_nivel
        if valor_base is not None:
            resultado = valor_base * multiplicador
            return min(max(rango_bajo, resultado), rango_alto)
            
    try:
        return float(val)
    except ValueError:
        pass
        
    return None

def procesar_columnas_dataframe(df, columnas):
    """
    Procesa las columnas especificadas en el dataframe.
    """
    for col in columnas:
        for i, val in enumerate(df[col]):
            try:
                valor_nivel = df.at[i, 'Level (PC=10)'] if 'Level (PC=10)' in df.columns else None
                resultado = procesar_valor_celda(val, valor_nivel)
                if resultado is not None:
                    df.at[i, col] = resultado
            except Exception as e:
                print(f"Error procesando fila {i}, columna {col}, valor {val}: {str(e)}")
                continue


# Procesar las nuevas columnas
columnas = ['Health (PC=10)', 'Magicka (PC=10)', 'Stamina (PC=10)']
procesar_columnas_dataframe(df, columnas)

# Mostrar los valores nulos
print("Conteo de valores nulos por columna:")
print(df.isnull().sum().sort_values(ascending=False))

# Procesar comerciantes de caballos
print("\nComerciantes de caballos:")
print(df.loc[df['Horses'].notnull()][['Name', 'Merchant', 'Horses']])
df.loc[df['Horses'].notnull(), 'Merchant'] = True


Conteo de valores nulos por columna:
Available          1015
Horses             1011
AdoptHF             993
StewardHF           982
Respawns            966
Training            958
Follower            955
Race Details        954
Services            950
Marry               948
Perks               904
Protected           901
Store               901
Buys                872
Gold                870
Merchant            859
Sells               859
Added by            835
Essential           760
House               751
Class Details       638
Location            384
Home City           339
Primary Skills      138
Faction(s)           41
Voice Type           39
Morality             38
Aggression           38
Stamina (PC=10)      31
Stamina              31
Health               14
Magicka              14
Health (PC=10)       14
Magicka (PC=10)      14
Class                13
Gender               13
RefID                 0
BaseID                0
Level                 0
Source                0
Nam

In [291]:
df.isnull().sum().sort_values(ascending=False)

Available          1015
Horses             1011
AdoptHF             993
StewardHF           982
Respawns            966
Training            958
Follower            955
Race Details        954
Services            950
Marry               948
Perks               904
Protected           901
Store               901
Buys                872
Gold                870
Sells               859
Merchant            854
Added by            835
Essential           760
House               751
Class Details       638
Location            384
Home City           339
Primary Skills      138
Faction(s)           41
Voice Type           39
Morality             38
Aggression           38
Stamina (PC=10)      31
Stamina              31
Health               14
Magicka              14
Health (PC=10)       14
Magicka (PC=10)      14
Class                13
Gender               13
RefID                 0
BaseID                0
Level                 0
Source                0
Name                  0
Race            

In [292]:
# Fillna for Horse Merchant
df.loc[df['Horses'].notnull()][['Name', 'Merchant', 'Horses']]

,Name,Merchant,Horses
101,Cedran,True,Black and white; 1000 gold
280,Hofgrir Horse-Crusher,True,Grey; 1000 gold
381,Geimund,True,Palomino; 1000 gold
478,Skulvar Sable-Hilt,True,Black; 1000 gold
548,Ulundil,True,Bay and white; 1000 gold
918,Agrane Peryval,True,Reindeer; 500+ gold


In [293]:
df.loc[df['Horses'].notnull(), 'Merchant'] = True

In [294]:
# Delete Avalilabe column
df.drop(columns=['Available'], inplace=True)

In [295]:
df['Morality'].unique()

array(['No Crime', nan, 'Any Crime', 'Violence Against Enemies',
       'Property Crime Only'], dtype=object)

In [296]:
df.loc[df['Class'].isnull()][['Name', 'Race', 'Class', 'Gender']]

,Name,Race,Class,Gender
91,Torom,Animals,NaN,NaN
103,Vigilance,Animals,NaN,NaN
193,Gratian Caerellius,Undead,NaN,NaN
195,Millius,Undead,NaN,NaN
274,Snippy,Animals,NaN,NaN
508,Tiber,Animals,NaN,NaN
509,Ysgramor,Animals,NaN,NaN
639,Stump,Animals,NaN,NaN
723,Maximian Axius,Undead,NaN,NaN
743,Bran,Animals,NaN,NaN


In [297]:
# Eliminar la columna 'Services'
df = df.drop(columns=['Services'])

# Definir listas de columnas para rellenar valores faltantes
fill_no = ['Essential', 'AdoptHF', 'StewardHF', 'Respawns', 'Follower', 'Marry', 'Protected']
fill_none = ['Training', 'Store', 'Buys', 'Sells', 'Class Details']
fill_indeterminate = ['Morality', 'Aggression', 'Class', 'Gender']

# Rellenar valores faltantes en la columna 'Added by'
df['Added by'] = df['Added by'].fillna('Vanilla')

# Rellenar valores faltantes en las columnas de fill_no
for col in fill_no:
    df[col] = df[col].fillna('No')

# Rellenar valores faltantes en las columnas de fill_none
for col in fill_none:
    df[col] = df[col].fillna('None')

# Rellenar valores faltantes en las columnas de fill_indeterminate
for col in fill_indeterminate:
    df[col] = df[col].fillna('Indeterminate')

In [298]:
df.isnull().sum().sort_values(ascending=False)

Horses             1011
Race Details        954
Perks               904
Gold                870
Merchant            854
House               751
Location            384
Home City           339
Primary Skills      138
Faction(s)           41
Voice Type           39
Stamina              31
Stamina (PC=10)      31
Magicka              14
Health               14
Health (PC=10)       14
Magicka (PC=10)      14
RefID                 0
Class                 0
Level                 0
Gender                0
Race                  0
Name                  0
Source                0
Aggression            0
Sells                 0
Training              0
Class Details         0
Essential             0
Morality              0
BaseID                0
Follower              0
Buys                  0
Respawns              0
Protected             0
AdoptHF               0
Store                 0
Added by              0
StewardHF             0
Marry                 0
Level (PC=10)         0
dtype: int64

In [300]:
df.to_csv('Skyrim_Named_Characters_Limpio.csv', index=False)